In [1]:
from regulus.utils import io
from regulus.topo.hmsc import * 
from regulus.tree.alg import *
from regulus.topo import persistence
from ipyregulus.tree import TreeView 
from ipyregulus import SidePanel
from ipywidgets import interact, interactive, FloatSlider

In [2]:
gauss = io.load('data/gauss4.p')

In [3]:
t1 = reduce(gauss.tree, filter=lambda n: persistence(n) > 0.1)

In [4]:
tv = TreeView(title='fiteness', field='lvl', tree=t1)
tv

TreeView(field='lvl', title='fiteness', tree=<regulus.tree.tree.Node object at 0x11a1813c8>)

In [5]:
tv.title = 'bar'

In [6]:
tv.attrs = dict(a='b')

In [7]:
tv.attrs

{'a': 'b'}

In [8]:
tv.attrs['c'] = 5

In [9]:
a = tv.attrs

In [10]:
tv.attrs = {}

In [11]:
tv.attrs = a

In [14]:
tree = gauss.tree

In [13]:
sim = io.load('data/simulations.p')

In [7]:
tree = sim.tree

In [15]:
class SafeNode(Node):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        if self.data is None:
            self.data = Partition(-1, 1)
        if not hasattr(self,'offset'):
            self.offset = 0

In [17]:
def persistence(n):
    if n.data is None:
        print('no data for node', n.__dict__)
        return 0
    if n.parent.data is None:
        print('no data for parent', n.parent.__dict__)
        return 0
    return n.parent.data.persistence - n.data.persistence

In [18]:
f = lambda n,v: persistence(n) > v
def update(view, v):
    view.tree = reduce(tree, filter=lambda n: f(n, v), factory=SafeNode)

In [19]:
def select(n):
    return {'persistence': persistence(n)}

In [20]:
tv = TreeView(title='lvl', field='lvl', tree=SafeNode())
sl = interactive(lambda v: update(tv,v), v=FloatSlider(min=0, max=1, step=0.01))

In [21]:
sp = SidePanel(title='lvl')
with sp:
    display(sl, tv)

In [ ]:
tv1 = TreeView(title='lvl', field='persistence', tree=reduce(tree), select=select)
sl1 = interactive(lambda v: update(tv1, v), v=FloatSlider(description='persistence', min=0, max=1, step=0.01))

In [22]:
sp1 = SidePanel(title='Persistence')
with sp1:
    display(sl1, tv1)

In [10]:
tv1.field = 'lvl'

In [11]:
sl1.f = lambda v: update(tv1, 1-v)

In [12]:
tv1.field = 'persistence'

In [12]:
from inspect import getsource
getsource(sl1.f)

'sl1 = interactive(lambda v: update(tv1, v), v=FloatSlider(min=0, max=1, step=0.01))\n'

In [16]:
from inspect import *

In [21]:
getsource(f)

'fp = lambda n, v: persistence(n) > v\n'

In [36]:
tv1

TreeView(field='persistence', title='lvl', tree=<regulus.tree.tree.Node object at 0x11a19fa90>)

In [37]:
sl1

interactive(children=(FloatSlider(value=0.0, description='persistence', max=1.0, step=0.01), Output()), _dom_c…

In [51]:
tv1.tree.parent.data.__dict__

{'id': -1,
 'hmsc': <regulus.topo.hmsc.HMSC at 0x115b52b70>,
 'persistence': 1,
 'span': [0, 0],
 'minmax_idx': [],
 'max_merge': False,
 '_x': None,
 '_y': None,
 'models': {},
 'measures': {}}

In [52]:
for n in tree:
    if n.data is None:
        print(n.__dict__)